<a href="https://colab.research.google.com/github/ambrosechoo999/BmiCalculator/blob/master/Next_WAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
!pip install pybaseball
from pybaseball import batting_stats

# **Downloading the data**

In [ ]:
START= 2002
END = 2023

In [ ]:
batting = batting_stats(START,END,qual =200) #qual is the number of plate appearances before they are included in dataset

In [ ]:
batting.to_csv("batting.csv") #Keep the stats into csv so no need repeated redownload the dataset

In [ ]:
batting = batting.groupby("IDfg",group_keys = False).filter(lambda x: x.shape[0]> 1)
#Filter out players with 1 season or less as the shape is 1
#group_ key = False so the index and id do not interefere each other and mixed

In [ ]:
batting

# **Creating a ML Target**

In [ ]:
def next_season(player):
  player = player.sort_values("Season")
  player["Next_WAR"] = player["WAR"].shift(-1) # shift moves the following year's WAR as the Next_WAR
  return player

batting = batting.groupby("IDfg",group_keys = False).apply(next_season)

In [ ]:
batting[["Name","Season","WAR","Next_WAR"]] # Notice there are NaN as there was not a following WAR to be used for Next_WAR

# **Cleaning** **Data**

In [ ]:
null_count = batting.isnull().sum() #see which column has alot of those NaN

In [ ]:
null_count

In [ ]:
complete_cols =list(batting.columns[null_count ==0]) # create a list of columns that do not have NaN or null values and are complete

In [ ]:
complete_cols

In [ ]:
batting = batting[complete_cols + ["Next_WAR"]].copy()

In [ ]:
batting # All NaN or null values are gone except for Next_WAR

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR
5828,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
5239,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
5497,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
1200,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,3.7,5.1
882,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,5.1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,27506,2023,Ha-seong Kim,SDP,27,152,538,626,140,100,...,99,85,117,106,82,424,0.233,0.293,4.1,NaN
1139,27676,2022,Vinnie Pasquantino,KCR,24,72,258,298,76,56,...,104,87,79,93,123,226,0.162,0.228,1.5,0.0
3504,27676,2023,Vinnie Pasquantino,KCR,25,61,231,260,57,31,...,102,88,87,91,121,202,0.179,0.253,0.2,NaN
2946,30116,2022,Seiya Suzuki,CHC,27,111,397,446,104,66,...,105,130,106,95,106,290,0.232,0.312,2.0,3.2


In [ ]:
batting.dtypes

In [ ]:
batting.dtypes[batting.dtypes == "object"] #ML only accept numbers not strings so these objects need to be converted like Names

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [ ]:
batting["Dol"] #Weird stat for assigning dollar to a player stat

In [ ]:
del batting["Dol"]

In [ ]:
batting["Age Rng"] #Age throughout the season

In [ ]:
del batting["Age Rng"]

In [ ]:
batting["team_code"] = batting["Team"].astype("category").cat.codes

In [ ]:
batting_full = batting.copy() # Makuing a copy because we going to drop any rows with Wars that are Nan and can use for forecast for 2003 or smthg like that
batting = batting.dropna() # drop all Nan rows

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha = 1)
split = TimeSeriesSplit(n_splits = 3)